In [1]:
import pandas as pd
df = pd.read_csv('D:/Corpus/NLP/empathetic dialogue/original_data(csv)/valid.csv',
                 encoding='ISO-8859-1')

In [2]:
df = df.drop(['prompt','´£¥Ü','µoÁn','intent','speaker_idx','emotion','selfeval','tags', 'utterance_idx','context'],axis=1)
df.head(6)

,conv_id,utterance
0,hit:3_conv:6,Today_comma_as i was leaving for work in the m...
1,hit:3_conv:6,Are you fine now?
2,hit:3_conv:6,Yeah_comma_i'm doing alright now_comma_ but wi...
3,hit:3_conv:6,Cool :) Is your car damaged a lot?
4,hit:3_conv:6,The car was badly damaged_comma_i veered outsi...
5,hit:4_conv:9,A few weeks ago_comma_ I was walking through m...


In [3]:
len(df)

12078

In [4]:
df.astype(str)

,conv_id,utterance
0,hit:3_conv:6,Today_comma_as i was leaving for work in the m...
1,hit:3_conv:6,Are you fine now?
2,hit:3_conv:6,Yeah_comma_i'm doing alright now_comma_ but wi...
3,hit:3_conv:6,Cool :) Is your car damaged a lot?
4,hit:3_conv:6,The car was badly damaged_comma_i veered outsi...
...,...,...
12073,hit:12361_conv:24722,tha is really cool what was your grade
12074,hit:12392_conv:24785,One of my coworkers has been arguing with his ...
12075,hit:12392_conv:24785,What are they arguing about?
12076,hit:12392_conv:24785,Everything and anything. It's annoying_comma_ ...


In [5]:
conv_id = df['conv_id'].iloc[0].split(':')[2]
df_a = []
df_b = []
last_context = None

turn=0
for n in range(len(df)):
    conv_id_n = df['conv_id'].iloc[n].split(':')[2]
    utterance_n =str(df['utterance'].iloc[n]).replace('_comma_',',').strip(' ')
    
    if conv_id == conv_id_n:
        turn +=1
        
        if turn % 2 == 0:
            b = utterance_n
            df_a.append(a + '<|endoftext|>')
            df_b.append(b + '<|endoftext|>')
        elif last_context is None:
            a = utterance_n
            last_context = utterance_n
        else:
            a = last_context + ' ' + utterance_n
            last_context = utterance_n
        
    else :
        turn = 1
        conv_id = conv_id_n
        a = utterance_n
        last_context = utterance_n

In [6]:
len(df_a)

5738

In [7]:
df_a[1]

"Today,as i was leaving for work in the morning,i had a tire burst in the middle of a busy road. That scared the hell out of me! Yeah,i'm doing alright now, but with minor injuries.<|endoftext|>"

In [8]:
with open('D:/論文閱讀/碩11008_郭旻學/Program/output/Transformer.txt','r',encoding="utf-8") as f:
    bad_sentence = f.read().splitlines()

In [9]:
bad_sentence[0].replace('[SEP]','')

'i would be so mad ! '

In [10]:
len(df_b)

5738

In [11]:
import random
sentence_a = []
sentence_b = []
label = []

for i in range(len(df_a)):
    # this is IsNextSentence
    sentence_a.append(df_a[i])
    sentence_b.append(df_b[i])
    label.append(0)

    # 50/50 whether is IsNextSentence or badSentence
    if random.random() >= 0.5:
        random_index = random.randint(0, len(df_b)-1)
        # this is NotNextSentence
        sentence_a.append(df_a[i])
        sentence_b.append(df_b[random_index])
        label.append(1)
    else:
        random_index = random.randint(0, len(bad_sentence)-1)
        # this is badSentence
        sentence_a.append(df_a[i])
        sentence_b.append(bad_sentence[random_index].replace('[SEP]','')+ '<|endoftext|>')
        label.append(1)

In [12]:
df_NSP = pd.DataFrame(list(zip(label, sentence_a, sentence_b)), columns =['label', 'sentence_a', 'sentence_b'])

In [13]:
df_NSP

,label,sentence_a,sentence_b
0,0,"Today,as i was leaving for work in the morning...",Are you fine now?<|endoftext|>
1,1,"Today,as i was leaving for work in the morning...","i know , i have no idea what i ' m a good idea..."
2,0,"Today,as i was leaving for work in the morning...",Cool :) Is your car damaged a lot?<|endoftext|>
3,1,"Today,as i was leaving for work in the morning...",i am glad you had a good time ! <|endoftext|>
4,0,"A few weeks ago, I was walking through my hall...","That's funny, hope he didn't give you a heart ..."
...,...,...,...
11471,1,One time I studied all night for my final exam...,What was her reaction to that<|endoftext|>
11472,0,One of my coworkers has been arguing with his ...,What are they arguing about?<|endoftext|>
11473,1,One of my coworkers has been arguing with his ...,I understand but memories are always nice.<|en...
11474,0,One of my coworkers has been arguing with his ...,That's so ridiculous!<|endoftext|>


In [14]:
df_NSP.iloc[100][1]

'I went to this thing called a Paint Night and I ended up making an okay looking painting<|endoftext|>'

In [15]:
df_NSP.iloc[100][2]

'That sounds fun, I was never any good at painting<|endoftext|>'

In [16]:
df_NSP.to_csv(r'D:/論文閱讀/碩11008_郭旻學/Program/EngagementClassifier_head/data/valid_user_system_conv.csv', index=False, header=False)